<a href="https://colab.research.google.com/github/antonum/Redis-Workshops/blob/main/02-Vector_Similarity_Search/02-RedisVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search with RedisVL



[Always-on demo](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook generates vector embeddings using pre-trained `sentence-transformers/all-MiniLM-L6-v2` model from HuggingFace, loads them to Redis and runs Vector Similarity search against Redis database.

In [5]:
#install Redis client and Hugging Face sentence transformers
!pip install -q sentence_transformers git+https://github.com/RedisVentures/redisvl.git@readme-enhancement

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.8 MB/s eta 0:00:00


Install Redis Stack locally

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to the Redis server

In [3]:
import os
import redis
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
#REDIS_PORT=12110
#REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"


In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from redisvl.utils.vectorize import HFTextVectorizer


# create a vectorizer
# choose your model from the huggingface website
hf = HFTextVectorizer(model="sentence-transformers/all-MiniLM-L6-v2")

# embed a sentence
test = hf.embed("This is a test sentence.")
test[:10]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.08429647237062454,
 0.057953670620918274,
 0.0044933767057955265,
 0.10582111030817032,
 0.007083416450768709,
 -0.01784469559788704,
 -0.016888044774532318,
 -0.01522829756140709,
 0.040473099797964096,
 0.03342254459857941]

In [10]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

"""from sentence_transformers import SentenceTransformer

from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
"""

tqdm.pandas()



### Embedding generation model

Here we are using `sentence-transformers/all-MiniLM-L6-v2` from HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2



In [23]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
test=model.encode("This is a test sentence.")
test[:]

array([ 8.42964724e-02,  5.79536706e-02,  4.49337671e-03,  1.05821110e-01,
        7.08341645e-03, -1.78446956e-02, -1.68880448e-02, -1.52282976e-02,
        4.04730998e-02,  3.34225446e-02,  1.04327641e-01, -4.70358171e-02,
        6.88472390e-03,  4.10179570e-02,  1.87119618e-02, -4.14923355e-02,
        2.36474089e-02, -5.65018356e-02, -3.36962305e-02,  5.09909801e-02,
        6.93032146e-02,  5.47841936e-02, -9.78840236e-03,  2.36971974e-02,
        1.99966021e-02,  9.71732661e-03, -5.88991679e-02,  7.30739301e-03,
        4.70264927e-02, -4.51007346e-03, -5.57997413e-02, -4.15944122e-03,
        6.47570640e-02,  4.80762385e-02,  1.70208160e-02, -3.18335509e-03,
        5.74023761e-02,  3.52318697e-02, -5.88394701e-03,  1.48329139e-02,
        1.15762977e-02, -1.07480727e-01,  1.91041995e-02,  2.20857169e-02,
        1.08645372e-02,  3.78196128e-03, -3.19403261e-02,  1.07278330e-02,
       -4.84231673e-03, -2.83362065e-02, -5.25735319e-02, -7.05867857e-02,
       -5.75558543e-02, -

Download 12k+ tweets

In [13]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-02-05 20:33:19--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv.1’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.01s   

2024-02-05 20:33:19 (217 MB/s) - ‘Labelled_Tweets.csv.1’ saved [2486081/2486081]



In [34]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
df=df.head(10) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
5,6,RT @Crypto____World: 🇿🇼ZIMBABWE🇿🇼\r\n\r\nCRYPT...
6,7,Pfizer Shares Acquired by Ipswich Investment M...
7,10,RT @NorthmanTrader: I repeat: The Fed is reckl...
8,12,S&amp;P 500 | $SPX | $SPX500 $USD\r\n\r\nlong ...
9,13,RT @TDANetwork: 📽️ #TheWatchList panel assesse...


### Generate Embeddings

Generate vector embeddings within the dataframe. This step can take 2-3 minutes on GPU runtime for all 12k records.

In [35]:
def text_to_embedding(text):
  return np.array(hf.embed(text), dtype=np.float32).tobytes()

#generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

  0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,full_text,text_embedding
0,1,@KennyDegu very very little volume. With $10T ...,b'*\x92\x81\xbd\x14h\x8b\xbdy\xdf\xe4\xbc\xc0\...
1,2,#ES_F achieved Target 2780 closing above 50% #...,b'Q\x1b\x02\xbd\x14~/\xbd\xa6z\xb1\xbcb\x99\xd...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...,b'\x10\xaa\xa3\xbd]}\x10\xbd\xc3\xe8\xb9=%\x08...
3,4,@Issaquahfunds Hedged our $MSFT position into ...,b'\xc4\x7f\xd1\xbc\xbe\n`\xbd79 =\xe4\xc0\xef=...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...,b'\xc6\r\x1e\xbdK\\\xd4\xbcH/\xa1\xbc\xebq7=\x...


### Create Helper Functions

- Save dataframe to Redis HASH
- Create RediSearch Index

In [27]:
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )



In [36]:
schema = {
    "index": {
        "name": "tweet:idx",
        "prefix": "tweet:",
        "storage_type": "hash", # default setting -- HASH
    },
    "fields": [
        {"name": "full_text", "type": "text"},
        {
            "name": "text_embedding",
            "type": "vector",
            "attrs": {
                "dims": 384,
                "distance_metric": "cosine",
                "algorithm": "HNSW",
                "datatype": "float32"
            }

        }
    ],
}

### Create index and load data to Redis

In [40]:
# clear Redis database (optional)
#redis.flushdb()

from redisvl.index import SearchIndex

index = SearchIndex.from_dict(schema)

# create Index
#create_redis_index(redis)

# load data from Dataframe to Redis HASH
#load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)
index.connect("redis://localhost:6379")

#keys = index.load(df.to_dict())
#df.to_dict()
keys = index.load(df.set_index('id').to_dict())


21:08:16 redisvl.index.index ERROR   Error while loading data to Redis
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/redisvl/index/index.py", line 493, in load
    return self._storage.write(
  File "/usr/local/lib/python3.10/dist-packages/redisvl/index/storage.py", line 222, in write
    self._validate(obj)
  File "/usr/local/lib/python3.10/dist-packages/redisvl/index/storage.py", line 405, in _validate
    raise TypeError("Object must be a dictionary.")
TypeError: Object must be a dictionary.


TypeError: Object must be a dictionary.

In [ ]:
#Check how the data is stored in Redis
!redis-cli $REDIS_CONN hgetall "tweet:1001"

1) "id"
2) "1001"
3) "text_embedding"
4) "\xd8\x1e\x03:\xdfb\xbd\xbcA\xb0\x94\xbc1K)<\xf4T\x94;%>\x11\xbd\xae3\r=\t\x98\x7f=\x99X\x88<uUR\xbd\xbc\xf6h\xbci\xb5\xd2\xbb\x15;\t\xbd\xf9m#<\xe7\xd4C=\x91\xc8\"\xbd\x1b\xa3-\xbd0^\t\xbdv\x00\xfd\xbc\xb6GE\xbdn\xadZ\xbc]\xae=\xbdH<\x99<\xf2b\xce\xbc\b`\xab<4DT\xbcmM\xd7\xbc\xdb\xf0\x82\xbb\x8e\xb0(=Gv\x8a\xbc\xd1\xa8M\xbdd,P=\bS\x86<\x8fj\xfa<??\xb5<\xd9[\xee\xbb\xcf\x8d\x90=\rW8=R\x81\x82\xbd\x96\xfb&<\xf2-:=Y\xe6\xe4\xbd\xae\x1d\x11\xbc\xd4\xb7\xac\xbd\xfbY<\xbbi\xc9\xcc\xbb\xf29b\xbcI\xe3\x90=0\x9f\xa7=\xef7\x0f=*\xbb\xbb\xbbP\xad\x17=\xb8\xaeE\xbd\xe2\x8e\x90=r\xd7\x02\xbd\xe0\x01\x9f\xbcv\xc3e;\a\xa5\xd8\xbc\x80{f<\x9bx\xaf\xbd\xf2\xc4\x89\xbcPo\x03=wB\x8e\xbc4\x91G;\xf8\xe6\x97\xba\xbd\xc1\x9a=\x12\xf7B\xbd\xb5\r\xca<_\xf2G\xbb\xbf\x0e\x86<\xb2r\x84<\xcd\xf7^\xbd5\xc9\xbe\xbd`K5\xbc\"\xe09\xbd\xc4\xd7==5y\xb3=\xad\x10\xb4\xbdr\xfc\x11\xbd\x11\x87O\xbc\xc9o<\xbd}\x87\x80\xbc\x1e\xa9\"\xbdV) \xbd\xe2\xfa\x85\xba\xc7y\x95=\xf2\xdf\xa6<\x03

## Query the database

[Alway-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Try queries like:
“Oil”, “Oil Reserves”, “Fossil fuels”

Lexical Full Text search quickly runs out of matches

Vector search continues to discover relevant tweets

In [ ]:
user_query="oil price"
# queries to try "oil reserve", "fossil fuels"

In [ ]:
#using Full Text Index
q = Query(user_query)\
  .return_fields("full_text")
res = redis.ft("tweet:idx").search(q)
if res.total==0:
  print("No matches found")
else:
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)

,id,full_text
0,tweet:3220,The relative performance of TIPS has historica...
1,tweet:311,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
2,tweet:1490,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
3,tweet:1585,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
4,tweet:1610,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
5,tweet:7189,Do higher oil prices help the consumer and sma...
6,tweet:636,Told you Saudi Arabia will bend the knee @jimc...
7,tweet:5405,https://t.co/3IJBXa5wuf Historic oil price plu...
8,tweet:5406,Historic oil price plunge trashes sector's pro...
9,tweet:3865,Today's book recommendation goes for the winne...


In [ ]:
#using Vector Similarity Index
query_vector=text_to_embedding(user_query)
q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
                .return_fields("result_score","full_text")\
                .dialect(2)\
                .sort_by("result_score", True)
res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
res_df

,id,result_score,full_text
0,tweet:444,0.369450807571,Would you spend $2 more a gallon of gasoline i...
1,tweet:11529,0.371095597744,RT @tradingcrudeoil: Crude oil closed up $0.48...
2,tweet:5654,0.381934285164,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
3,tweet:204,0.396132171154,Bad news for #oil. It’s going to between $10 ...
4,tweet:9189,0.409308671951,Oil erases gains for the day in fall to $25 ht...
5,tweet:7189,0.429816782475,Do higher oil prices help the consumer and sma...
6,tweet:9330,0.430081248283,The price of Texas intermediate oil (WTI) slum...
7,tweet:531,0.431391596794,#OIL Sentiment ($22.50)\r\n\r\nWhat’s next for...
8,tweet:178,0.441844820976,OH how bullish for #oil LOL\r\n\r\n#OOTT #Oi...
9,tweet:6867,0.442979931831,$DXY 99.55-0.57%&lt;==US Dollar lower #Fed $2....
